In [ ]:
import polars as pl
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

sys.path.insert(0, str(Path.cwd().parent / "src"))

from db.config import get_database_url

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
)

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Cada linha
# Winner 0 = t1 win
# Winner 1 = t2 win
# History_Size: amount of previous matches to consider for average opening kills/deaths
# t1_id, t1_avg_opening_kills, t1_avg_opening_deaths, t2_id, t2_avg_opening_kills, t2_avg_opening_deaths, t1_old1_match_avg_opening, winner

In [29]:
db_url = get_database_url(make_ipv6_in_bracket=True)

player_map_stats = pl.read_database_uri(
    """
    SELECT * FROM player_map_stats
                                        """,
    db_url,
)
player_map_stats

map_stat_id,player_id,team_id,opening_kills_ct,opening_deaths_ct,multikills_ct,kast_ct,clutches_ct,kills_ct,headshot_kills_ct,assists_ct,flash_assists_ct,deaths_ct,traded_deaths_ct,adr_ct,swing_ct,rating_3_dot_0_ct,opening_kills_tr,opening_deaths_tr,multikills_tr,kast_tr,clutches_tr,kills_tr,headshot_kills_tr,assists_tr,flash_assists_tr,deaths_tr,traded_deaths_tr,adr_tr,swing_tr,rating_3_dot_0_tr
i32,i32,i32,i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]",i32,i32,i32,"decimal[38,10]",i32,i32,i32,i32,i32,i32,i32,"decimal[38,10]","decimal[38,10]","decimal[38,10]"
215707,19617,11446,1,0,4,80.0000000000,1,17,5,2,1,6,0,151.4000000000,17.2300000000,3.0000000000,1,1,4,75.0000000000,0,12,6,5,1,6,2,107.9000000000,10.0900000000,1.9700000000
215707,22649,11446,1,5,2,60.0000000000,0,5,2,2,0,8,1,52.0000000000,-6.8300000000,0.5500000000,2,1,3,91.7000000000,1,12,4,3,0,6,3,95.0000000000,5.5900000000,1.5700000000
215707,21065,11446,0,0,1,80.0000000000,0,7,1,2,0,8,2,72.7000000000,-4.7600000000,0.7200000000,1,1,2,83.3000000000,0,9,1,3,0,5,1,70.1000000000,2.4500000000,1.2000000000
215707,21651,11446,0,3,0,60.0000000000,0,2,2,3,0,9,2,39.5000000000,-4.4000000000,0.4000000000,0,2,1,75.0000000000,0,9,5,2,0,8,1,81.9000000000,-1.4200000000,1.0500000000
215707,21650,11446,0,0,0,60.0000000000,0,4,2,0,0,10,3,41.7000000000,-5.1700000000,0.5400000000,1,2,1,83.3000000000,0,6,1,3,0,6,1,54.4000000000,1.1900000000,0.9800000000
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
215735,7528,4991,0,1,2,91.7000000000,1,9,4,9,0,6,2,88.8000000000,0.4100000000,1.1500000000,0,0,3,90.0000000000,0,10,4,5,0,7,2,110.8000000000,-0.6900000000,1.6700000000
215735,23046,4991,1,0,5,91.7000000000,0,15,5,1,0,5,0,95.7000000000,8.6900000000,1.7100000000,2,0,2,90.0000000000,0,9,6,8,4,6,1,83.3000000000,5.4700000000,1.5300000000
215735,17803,4991,0,4,0,83.3000000000,0,4,2,7,0,7,1,67.7000000000,-6.3900000000,0.5900000000,0,2,2,70.0000000000,0,6,2,4,2,8,2,61.1000000000,-3.4000000000,1.0100000000
